# About Unit 1

Welcome to the Marquette University AIM time series analysis curriculum! In this first unit you will be introduced to time series analysis through a series of projects. We will explore what a time series is, the components that make it up, and introduce the concept of exploratory data analysis.

# Getting Started

**Import Packages**

Run the following code to bring the necessary packages into your environment. Ensure you are running a python kernel >=3.0.0.

In [ ]:
!pip install distfit

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats import descriptivestats
from statsmodels.graphics.tsaplots import plot_acf
from distfit import distfit

# Load a Sample Dataset

Before we start, we need data to look at! Thankfully, the statsmodels package gives us several sample data sets to play with. We are going to be using the Sea Surface Temperature dataset for this project. Run the following code to load the time series.

In [ ]:
df = sm.datasets.elnino.load_pandas().data
df.set_index('YEAR', inplace=True)

temp_array = df.to_numpy().flatten()
date_range = pd.date_range(start='1950-01-01', end='2010-12-31', freq='M')

plt.plot(date_range, temp_array)
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.title('Sea Surface Temperature')
plt.show()

# The Time Series

What is a time series? A time series is a sequence of discrete data that is shown as a function of time. Many common things we use every day are time series, and many other sets of data can be shown as a time series. Stock market charts, for example, are a time series.

**Components of a Time Series**

A time series can be thought of as the combination of several components. The core 3 components that make up a time series are the trend, seasonality, and residuals. When these three components are mixed together, you will see the original series.

**Trend**

The trend of a time series is the underlying direction and magnitude of the data. We use trend in our every day lives to discuss what certain charts look like. For instance, one might say that your grade in this class is trending down.

**Seasonality**

The seasonality of a time series describes patterns in the data that repeat, and the degree to which they repeat. Think about temperature fluctuations during the year. They predictably rise in the summer and fall in the winter, and if plotted, would look similar to a sine wave.

**Residuals**

Residuals are everything not captured by seasonality or trend. This is the noise of the time series and accounts for things like random variability or other components. In a perfect decomposition, there are no residuals. This means that the trend and seasonality can completely describe the time series.

# Decomposing a Time Series

When we analyze a time series by breaking it down into its components, we are *decomposing* the data. We want to be able to see the long term trend of the series and if there is any seasonality that we can capture. If our model is good enough, we can accurately forecast future data using these components. There are many ways to decompose a time series, but we will start with a naive decomposition. In a naive decomposition there are two main methods: additive or multiplicative.

**Naive Decomposition**

Naive Decomposition works by using a moving average. The trend is estimated with the moving average, and then subtracted from the series. The seasonality is then calculated by averaging the detrended data for each season, and the residuals are found by subtracting or dividing each component out of the original time series.

**Additive Decomposition**

The additive model follows the equation $Y(t) = T(t) + S(t) + R(t)$, where $Y(t)$ is the observed value at time $t$, $T(t)$ is the trend, $S(t)$ is the seasonality, and $R(t)$ is the residual. Additive decomposition is used when variation in seasonality is stable over time. Residuals should be centered on 0.

**Multiplicative Decomposition**

The multiplicative model is similar to the additive, but instead of adding we are multiplying the components together. This yields the formula $Y(t) = T(t) * S(t) * R(t)$. Multiplicative decomposition is used when variation in seasonality changes over time. Residuals should be centered on 1.

**Decomposing Our Data**

Which model should we use for our data? The best way is to try both and compare the residuals. We can use the seasonal_decompose() function within the statsmodels package to accomplish this. Because our data is monthly, we set the period to 12. This will allow us to find seasonality in terms of a year.

In [ ]:
add_decomp = seasonal_decompose(temp_array, model='additive', period=12)
add_decomp.plot()

In [ ]:
mul_decomp = seasonal_decompose(temp_array, model='multiplicative', period=12)
mul_decomp.plot()

**Analyzing the Result**

After running our code, we can clearly see that the multiplicative model seems to be better suited for our data. The residuals are nicely centered around 1, and appear to be randomly distributed. We want the residuals to be randomly distributed because that means the decomposition has accounted for all of the structure. However, just because the points look random does not mean they actually are. We need to check the residuals for autocorrelation using an autocorrelation plot.

**Autocorrelation**

Autocorrelation is a feature of a discrete time series that can be defined as the degree of correlation that exists between different values in the same series. When a data set is completely random, there is no autocorrelation. In other words, if a series is autocorrelated we could predict future values based off of previous ones.

**The Autocorrelation Function**

The autocorrelation function is used to determine the autocorrelation of a series. This function is defined as $\hat{R}(k) = \frac{1}{(n - k)\sigma^2} \sum_{t=1}^{n-k} (X_t - \mu)(X_{t+k} - \mu)$, where $k$ is the lag value. It measures the linear relationship between an observation at time $t$ and the observation at time $t + k$.

We can easily calculate the autocorrelation and plot it within python using the statsmodels plot_acf() function.

In [ ]:
residuals = mul_decomp.resid[~np.isnan(mul_decomp.resid)]
plot_acf(residuals)

**Reading the ACF Plot**

The ACF plot shows the correlation between observations of a time series separated by various lag intervals. The $x$ axis shows the number of lags, while the $y$ axis represents the autocorrelation coefficients. The closer the autocorrelation coefficient is to $1$ or $-1$, the more autocorrelated it is. The value at $x = 0$ is always $1$, because no lag is applied. The shaded blue region represents the confidence interval of the plot. If values fall in this region, they can be treated as if they are uncorrelated.

Additionally, different patterns on an ACF plot can show information critical to tuning predictive models. This will be covered later in the course when we discuss ARIMA models, so keep this in mind.

Unfortunately it looks as though there is autocorrelation in the series, meaning the decomposition didn't take all patterns out of the data. This can be expected since the naive decomposition is not very robust.

# Exploratory Data Analysis

Exploratory data analysis is an approach to analyze data through the use of graphs and other visualization methods. EDA is not so much a set technique, but rather a different attitude to take when approaching a new problem. The purpose of EDA is to allow statisticians to understand the data they are working with so that they can select appropriate models, suggest hypotheses, or discover previously hidden characteristics of the data.

**Exploring the Sea Surface Temperature Dataset**

Let's use some EDA techniques on the previous time series to learn more about it! We should start by checking for the presence of outliers. Outliers can "dirty" our data, preventing us from fitting a distribution or forecasting future values. If outliers exist, we need to remove them before future modelling. We can do this using a box plot. On a box plot, outliers are shown as dots.

In [ ]:
plt.boxplot(temp_array)
plt.title('Sea Surface Temperature')
plt.show()

**Interpreting the Box Plot**

Surprisingly it appears that we have no outliers. Because of EDA, we can hypothesize that this is a result of the Earth's temperature being relatively regulated. This is great news and means that we can proceed with exploring the data!

**Descriptive Statistics**

Now that we know we have clean data, we should look at the broad statistics of the time series, which we will do using descriptive statistics. Descriptive statistics are all of the basic statistics of a series, such as the mean, median, kurtosis, etc. We can do this using statsmodels and the following code.

In [ ]:
descriptivestats.describe(temp_array)

**Understanding Descriptive Statistics**

The statsmodels package provides a long list of descriptive statistics, some of which you may or may not be familiar with. We will be covering what each value means, as they are important in exploring the data you are working with.

**nobs**

Nobs stands for number of observations in the statsmodels package. You may see it referred to as $n$ in a statistics textbook. This just means the number of items in your sample.

**missing**

Missing represents the number of missing values in our series. Because we have none, the number is 0. If there were any NaN values in our series, this would count their frequency.

**mean**

The arithmetic mean, as described by $\mu = \frac{1}{n} \sum_{i=1}^{n} x_i$.

**std_err**

This is the standard error of the mean, assuming no correlation. The standard error is the standard deviation of a sample divided by the square root of $n$. Its formula can be described as $SE = \frac{\sigma}{\sqrt{n}}$. Standard error of the mean is an inferential statistic that describes the variability of the sample mean, and how closely it is to the population mean.

**upper/lower ci**

This is the upper and lower confidence interval of the data, respectfully. Statsmodels allows you to choose whether to use a normal or t distribution for the confidence intervals, as well as to choose the alpha level. The default alpha value is 0.05 giving a 95% confidence interval.

**std**

Std is the standard deviation of the series, which measures the amount of variability in values around the sample's mean. The standard deviation is represented by the formula $\sigma = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2}$.

**iqr/iqr_normal**

This stands for the interquartile range. The interquartile range is a measure of dispersion within our data. IQR shows the difference between the 75th and 25th percentiles to give the range of the middle 50% of our data. On a box plot, the box itself represents the IQR. The iqr_normal statistic shows us the IQR of the data against a normal distribution.

**mad/mad_normal**

The MAD is the mean absolute deviation. This is the average distance between each data point and the mean. It's formula can be described as $\text{MAD} = \frac{1}{n} \sum_{i=1}^{n} \left| x_i - \mu \right|$. Once again, the mad_normal represents the mean absolute deviation relative to a normal distribution.

**coef_var**

This is the coefficient of variation, which is a standardized measure of dispersion within our dataset. It's formula is described by $\text{CV} = \frac{\sigma}{\mu}$. As you can see, this is the ratio of the standard deviation to the mean.

**range**

The range is the distance between the largest and smallest values in our sample. It describes the range of values that are encompassed by the dataset.

**max/min**

The max and min are the largest and smallest values in the set, respectfully.

**skew**

The skew is the measure of asymmetry of the distribution. Positive skews show a right skew, while negative skews mean a left skew. This can be respresented as the formula $\text{Skewness} = \frac{\frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^3}{\left( \frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2 \right)^{3/2}}$.

**kurtosis**

Kurtosis is a measure of the tailedness of a distribution. This means that it measures the frequency of outliers present in the sample. A normal distribution has a kurtosis of 0, for example. There are many types of and ways to calculate kurtosis, but the statsmodels package uses the 4th moment of the mean, which can be described as $\text{Kurtosis} = \frac{\frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^4}{\left( \frac{1}{n} \sum_{i=1}^{n} (x_i - \mu)^2 \right)^2} - 3$.

**jarque_bera**

This is the Jarque-Bera test statistic for normality. It is based on skewness and kurtosis, and is a goodness-of-fit test of whether or not the sample data follows a normal distribution. If the value is not close to 0, it means that the distribution of the data is nonnormal.

**mode/mode_freq**

The mode is the most frequently occuring value in the distribution. Likewise, mode_freq is the frequency of the modal value.

**median**

The median is one way to describe the average of the dataset, and is much less vulnerable to being skewed by outliers than the mean. The median is the central number of a sample if you were to arrange every point in order.

**percentiles**

The percentiles shown by statsmodels describe the percentiles of the values in the sample. Each percentile describes the upper bound of values to be in that percentile. For example, if the 5th percentile was 10, we know that the 5% lowest values of the sample are $\leq 10$.

**Looking at the Distribution**

Looking at the results of our descriptive stats, it appears that our data is not normally distributed. We should also learn how our data is distributed, since this is crucial for model selection later on. We can roughly see what kind of distribution our data possesses using a histogram.

In [ ]:
plt.hist(temp_array, bins=25)
plt.title('Sea Surface Temperature')
plt.show()

Our data appears to be quite oddly distributed, possibly across some type of bimodal distribution. We could guess the two peaks may be the result of extremes in the summer and winter. Many models require data to be distributed across a normal distribution, so now we know we would either have to transform our data or choose alternative forecasting methods. Without using EDA techniques, we would not have been able to discover this!

# Conclusion

Hooray, you made it! This concludes Unit 1 of the AIM Time Series Analysis curriculum. After completing this, you should have a good foundational grasp on what a time series is, its basic components, and an idea of how to approach analysis through EDA. Future units will dive deeper into the modelling of these series, and by the end, you will be a master of analyzing this type of data.